In [28]:
#| default_exp reading
%load_ext autoreload
%autoreload 2

import sys
from fastcore.xtras import Path


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
#| hide
# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

# Leitura
> Este módulo concentra funções para a leitura dos diversos arquivos que compõe a base de dados

* `STEL` - Serviços Privados de Telecomunicações
* `RADCOM` - Serviço de Radiodifusão Comunitária
* `MOSAICO` - Demais serviços de Radiodifusão, e.g. TV, RTV, FM, AM, etc.
* `AERONAUTICA` - Consolidação de diversas bases de dados públicas da aeronáutica 

In [30]:
#| export
from pathlib import Path
from typing import Union

import pandas as pd
from dotenv import find_dotenv, load_dotenv



load_dotenv(find_dotenv())

True

## MOSAICO
> O mosaico atualmente é composto por 2 bases complementares originárias de um banco `MongoDB`: 

-  `Estações`
- `Plano Básico`

In [31]:
#|eval: false
#|code_fold: true
folder = Path.cwd().parent / 'extracao' / 'datasources' / 'arquivos' / 'saida'

In [32]:
#|eval: false
#|code_fold: true

def read_df(name: str)->pd.DataFrame:
    return pd.read_parquet(folder / f'{name}.parquet.gzip', dtype_backend='pyarrow')# .astype("category")

In [33]:
df = read_df('estacoes')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 975391 entries, 0 to 975390
Data columns (total 29 columns):
 #   Column                       Non-Null Count   Dtype                                                       
---  ------                       --------------   -----                                                       
 0   Frequência                   975391 non-null  double[pyarrow]                                             
 1   Entidade                     975391 non-null  string[pyarrow]                                             
 2   Fistel                       975391 non-null  uint64[pyarrow]                                             
 3   Serviço                      975391 non-null  uint16[pyarrow]                                             
 4   Estação                      975391 non-null  string[pyarrow]                                             
 5   Latitude                     975391 non-null  float[pyarrow]                                        

In [37]:
df.Fonte.unique()

<ArrowExtensionArray>
[                          'MOSAICO-LICENCIAMENTO',
                                          'AISGEO',
                  'MOSAICO-LICENCIAMENTO | AISGEO',
                                            'STEL',
                                     'MOSAICO-SRD',
                  'MOSAICO-LICENCIAMENTO | AISWEB',
                                             'SRD',
                                   'ICAO | AISWEB',
                    'MOSAICO-LICENCIAMENTO | ICAO',
           'MOSAICO-LICENCIAMENTO | ICAO | AISWEB',
                                            'ICAO',
                                          'AISWEB',
                                   'ICAO | AISGEO',
           'MOSAICO-LICENCIAMENTO | ICAO | AISGEO',
                            'STEL | ICAO | AISWEB',
                                   'STEL | AISWEB',
                                     'STEL | ICAO',
                         'CANALIZACAO-VOR/ILS/DME',
 'MOSAICO-LICENCIAMENTO | CANALIZACAO-VOR/

In [40]:
c = df.Fonte.isin(['ICAO | AISWEB', 'ICAO | AISGEO', 'ICAO', 'AISWEB', 'AISGEO', 'REDEMET', 'RADAR'])
df[c & (df.Estação != '-1')]

,Frequência,Entidade,Fistel,Serviço,Estação,Latitude,Longitude,Código_Município,Município,UF,...,Potência_Transmissor(W),Ganho_Antena(dBd),Ângulo_Elevação_Antena,Azimute_Antena,Altura_Antena(m),Atenuação_Linha(db/100m),Perdas_Acessórias_Linha(db),Padrão_Antena(dBd),Comprimento_Linha(m),Relatório_Canal
683,0.200,NDB - CABO FRIO,1030052263,19,1557637,-22.923334,-42.071499,3300704,Cabo Frio,RJ,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1
699,0.205,NDB - MAXARANGUAPE,50014083426,108,684424649,-5.386167,-35.530998,2410405,Pureza,RN,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1
713,0.210,NDB - ARAXA,1030052263,19,859761,-19.561001,-46.964668,3104007,Araxá,MG,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1
776,0.230,NDB - TABATINGA NDB NOT AVBL BEYOND 50 NM,11030012059,19,355305,-4.249333,-69.936165,1304062,Tabatinga,AM,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1
816,0.240,NDB - TIRIOS NDB NOT AVBL BEYOND 50 NM,1030052263,19,623299550,2.219028,-55.941799,1505106,Óbidos,PA,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
638018,1185.000,DME - ZUMBI 98X,50444239804,108,1000822831,-9.509894,-35.786873,2704302,Maceió,AL,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1
638020,1185.000,DME - CACHIMBO 98X,50444239804,108,1000822831,-9.331328,-54.965038,1505031,Novo Progresso,PA,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1
638037,1193.000,DME - GOIANIA 106X,50444239804,108,1000822904,-16.640833,-49.211113,5208707,Goiânia,GO,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1
638039,1193.000,DME - SAO GONCALO DO AMARANTE 106X,50444239804,108,1000822904,-5.771811,-35.368679,2412005,São Gonçalo do Amarante,RN,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1


In [ ]:
df

,Frequência,Entidade,Fistel,Serviço,Estação,Latitude,Longitude,Código_Município,Município,UF,...,Potência_Transmissor(W),Ganho_Antena(dBd),Ângulo_Elevação_Antena,Azimute_Antena,Altura_Antena(m),Atenuação_Linha(db/100m),Perdas_Acessórias_Linha(db),Padrão_Antena(dBd),Comprimento_Linha(m),Relatório_Canal
2092,161.25,STOCKTOTAL TELECOMUNICACOES LTDA,50405348339,011,690851588,-23.634281,-46.69974,3550308,São Paulo,SP,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
31300,599.0,RADIO E TV UMBU LTDA,50409595837,801,1002346697,-27.96611,-52.906944,4320107,Sarandi,RS,...,100.0,11.76,4,310.0,68.35,2.88,0.6,"{'0': 4.961, '10': 5.058, '20': 5.348, '30': 5...",90,http://sistemas.anatel.gov.br/se/eApp/reports/...
28932,57.0,MUNICIPIO DE SAO JOSE DO DIVINO,04023534471,800,322384893,-18.478611,-41.389446,3163300,São José do Divino,MG,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.5,<NA>,<NA>,http://sistemas.anatel.gov.br/se/eApp/reports/...
29220,177.0,TELEVISAO CIDADE VERDE S/A,50400805227,800,323715311,-16.856112,-53.0325,5101209,Araguainha,MT,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"{'0': 0.915, '10': 0.819, '20': 0.264, '30': 0...",<NA>,http://sistemas.anatel.gov.br/se/eApp/reports/...
22402,207.0,MUNICIPIO DE ALFENAS,50400367360,800,322378494,-21.429167,-45.947224,3101607,Alfenas,MG,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.5,<NA>,<NA>,http://sistemas.anatel.gov.br/se/eApp/reports/...


In [ ]:
df.Multiplicidade.fillna('1', inplace=True)

In [ ]:
df.replace('', '-1', inplace=True)

d:\Applications\Scoop\apps\miniconda3\current\envs\dados\lib\site-packages\pandas\core\arrays\arrow\array.py:480: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result[valid] = op(np.array(self)[valid], other)


In [ ]:
df.loc[df.Estação == '1007550390', ['Frequência', 'Entidade', 'Latitude', 'Longitude', 'Fonte']]

AttributeError: 'DataFrame' object has no attribute 'Estação'

In [ ]:
for c in df.columns:
    if c not in ['Frequência', 'Latitude', 'Longitude']:
        # df[c] = df[c].str.replace('', '-1')
        df[c] = df[c].fillna('-1')
        df[c] = df[c].astype('category')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 974142 entries, 0 to 974141
Data columns (total 30 columns):
 #   Column                       Non-Null Count   Dtype          
---  ------                       --------------   -----          
 0   Id                           974142 non-null  category       
 1   Frequência                   974142 non-null  double[pyarrow]
 2   Entidade                     974142 non-null  category       
 3   Fistel                       974142 non-null  category       
 4   Número_Estação               974142 non-null  category       
 5   Latitude                     974142 non-null  float[pyarrow] 
 6   Longitude                    974142 non-null  float[pyarrow] 
 7   Código_Município             974142 non-null  category       
 8   Município                    974142 non-null  category       
 9   UF                           974142 non-null  category       
 10  Classe                       974142 non-null  category       
 11  Num_Serviço  

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 974142 entries, 1 to 974142
Data columns (total 29 columns):
 #   Column                       Non-Null Count   Dtype          
---  ------                       --------------   -----          
 0   Frequência                   974142 non-null  double[pyarrow]
 1   Entidade                     974142 non-null  category       
 2   Fistel                       974142 non-null  category       
 3   Serviço                      974142 non-null  category       
 4   Estação                      974142 non-null  category       
 5   Latitude                     974142 non-null  float[pyarrow] 
 6   Longitude                    974142 non-null  float[pyarrow] 
 7   Código_Município             974142 non-null  category       
 8   Município                    974142 non-null  category       
 9   UF                           974142 non-null  category       
 10  Classe                       974142 non-null  category       
 11  Classe_Emissão    

In [ ]:
df.rename(columns={'Altura_Estação': 'Cota_Base_Torre(m)',
                   'Número_Estação': 'Estação',
                   'Num_Serviço': 'Serviço' 
                   }, inplace=True)

In [ ]:
from extracao.constants import COLS_SRD

df = df[COLS_SRD]

In [ ]:
df.to_parquet(folder / 'estacoesV3.parquet.gzip', compression='gzip', engine='pyarrow')

In [ ]:
df[df.Fistel == '50011346337']

,Frequência,Entidade,Fistel,Serviço,Estação,Latitude,Longitude,Código_Município,Município,UF,...,Potência_Transmissor(W),Ganho_Antena(dBd),Ângulo_Elevação_Antena,Azimute_Antena,Altura_Antena(m),Atenuação_Linha(db/100m),Perdas_Acessórias_Linha(db),Padrão_Antena(dBd),Comprimento_Linha(m),Relatório_Canal
34994,105.9,ASSOCIACAO DE MORADORES DA PRAIA DO CANTO,50011346337,231,631394656,-20.298056,-40.295277,3205309,Vitória,ES,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [ ]:
smp = read_df('smp')
smp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239937 entries, 0 to 239936
Data columns (total 28 columns):
 #   Column                Non-Null Count   Dtype   
---  ------                --------------   -----   
 0   Frequência            239937 non-null  category
 1   Entidade              239937 non-null  category
 2   Fistel                239937 non-null  category
 3   Número_Estação        239937 non-null  category
 4   Município             209059 non-null  category
 5   Código_Município      239937 non-null  category
 6   UF                    239937 non-null  category
 7   Latitude              239937 non-null  category
 8   Longitude             239937 non-null  category
 9   Classe                239937 non-null  category
 10  Num_Serviço           239937 non-null  category
 11  Classe_Emissão        239937 non-null  category
 12  Largura_Emissão(kHz)  239937 non-null  category
 13  Validade_RF           239937 non-null  category
 14  Status                239937 non-nul

In [ ]:
smp.iloc[:, 17:].sample(5)

,Potência(W),Cod_Tipo_Antena,Polarização,Ganho_Antena,FC_Antena,Ang_MP_Antena,Ângulo_Elevação,Azimute,Altura_Antena,Perdas_Acessorias,Log
100275,40.0,760,X,14.27,25.0,65.0,0.0,280.0,16.0,0.0,"[(""Colunas"", ['Código_Município', 'Fistel', 'F..."
155682,32.5,760,X,14.3,26.0,82.0,0.0,355.0,90.0,0.0,"[(""Colunas"", ['Código_Município', 'Fistel', 'F..."
110093,40.0,760,X,11.96,25.0,74.0,0.0,230.0,60.0,0.0,"[(""Colunas"", ['Código_Município', 'Fistel', 'F..."
96501,40,760,X,13.5,25.0,63.0,3,210,34,0.00,"[(""Colunas"", ['Código_Município', 'Fistel', 'F..."
145489,74.0,760,X,15.4,27.0,84.0,2.0,260.0,88.0,0.0,"[(""Colunas"", ['Código_Município', 'Fistel', 'F..."


In [ ]:
anatel = read_df('anatel')
anatel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 974088 entries, 0 to 974087
Data columns (total 18 columns):
 #   Column                Non-Null Count   Dtype   
---  ------                --------------   -----   
 0   Frequência            974088 non-null  category
 1   Entidade              974086 non-null  category
 2   Fistel                971380 non-null  category
 3   Número_Estação        971380 non-null  category
 4   Município             974088 non-null  category
 5   Código_Município      974088 non-null  category
 6   UF                    974088 non-null  category
 7   Latitude              974088 non-null  category
 8   Longitude             974088 non-null  category
 9   Classe                970955 non-null  category
 10  Num_Serviço           971380 non-null  category
 11  Classe_Emissão        965996 non-null  category
 12  Largura_Emissão(kHz)  970959 non-null  category
 13  Validade_RF           966103 non-null  category
 14  Status                971380 non-nul

In [ ]:
anatel.Fonte.value_counts()

Fonte
MOSAICO                                          827673
CANALIZACAO-SMP                                  118425
STEL                                              20319
SRD                                                4963
ICAO                                               1610
ICAO | AISWEB                                       264
AISGEO                                              219
ICAO-CANALIZACAO-VOR                                167
RADARES                                             152
AISWEB                                              103
AISWEB-CANALIZACAO-VOR                               78
ICAO-CANALIZACAO-VOR | AISWEB-CANALIZACAO-VOR        45
ICAO | AISGEO                                        44
REDEMET                                              26
Name: count, dtype: int64